In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import arules
from arules.utils import five_quantile_based_bins, top_bottom_10, top_5_variant_variables
import csv

from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
np.random.seed(12345)
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
import math
binary = CountVectorizer(binary=True)

data = pd.read_csv("crash.csv")
data.head()

,State,Atmospheric Condition,Crash Date,Fatalities in crash,Roadway,Age,Alcohol Results,Person Type,Drug Involvement,Race,Gender,Injury Severity,Crash Date.year,Crash Date.month,Crash Date.day-of-month,Crash Date.day-of-week
0,Alaska,Clear,1/5/11,1,Rural-Principal Arterial-Interstate,27.0,0.0,Driver of a Motor Vehicle In-Transport,Yes,NaN,Male,Non-incapacitating Evident Injury (B),2011,1,5,3
1,Alaska,Clear,1/5/11,1,Rural-Principal Arterial-Interstate,60.0,0.0,Driver of a Motor Vehicle In-Transport,No,White,Female,Fatal Injury (K),2011,1,5,3
2,Arizona,Clear,1/1/11,1,Urban-Other Principal Arterial,24.0,NaN,Passenger of a Motor Vehicle In-Transport,Not Reported,NaN,Female,No Injury (O),2011,1,1,6
3,Arizona,Clear,1/1/11,1,Urban-Other Principal Arterial,27.0,0.0,Driver of a Motor Vehicle In-Transport,No,NaN,Male,No Injury (O),2011,1,1,6
4,Arizona,Clear,1/1/11,1,Urban-Other Principal Arterial,82.0,0.0,Pedestrian,No,Unknown,Female,Fatal Injury (K),2011,1,1,6


In [2]:
#Get rid of the date columns - since crash date is already provided 
del data['Crash Date.year']
del data['Crash Date.month']
del data['Crash Date.day-of-month']
data.head()

,State,Atmospheric Condition,Crash Date,Fatalities in crash,Roadway,Age,Alcohol Results,Person Type,Drug Involvement,Race,Gender,Injury Severity,Crash Date.day-of-week
0,Alaska,Clear,1/5/11,1,Rural-Principal Arterial-Interstate,27.0,0.0,Driver of a Motor Vehicle In-Transport,Yes,NaN,Male,Non-incapacitating Evident Injury (B),3
1,Alaska,Clear,1/5/11,1,Rural-Principal Arterial-Interstate,60.0,0.0,Driver of a Motor Vehicle In-Transport,No,White,Female,Fatal Injury (K),3
2,Arizona,Clear,1/1/11,1,Urban-Other Principal Arterial,24.0,NaN,Passenger of a Motor Vehicle In-Transport,Not Reported,NaN,Female,No Injury (O),6
3,Arizona,Clear,1/1/11,1,Urban-Other Principal Arterial,27.0,0.0,Driver of a Motor Vehicle In-Transport,No,NaN,Male,No Injury (O),6
4,Arizona,Clear,1/1/11,1,Urban-Other Principal Arterial,82.0,0.0,Pedestrian,No,Unknown,Female,Fatal Injury (K),6


In [3]:
#rename columns
data.rename(columns = {'Atmospheric Condition': 'Atmospheric_Condition', 'Crash Date' : 'Crash_Date', 
            'Fatalities in crash' : 'Crash_Fatalities', 'Alcohol Results': 'Alcohol_Results', 
            'Person Type' : 'Person_Type', 'Drug Involvement' : 'Drug_Involvement', 
            'Injury Severity' : 'Injury_Severity', 'Crash Date.day-of-week': 'Dayo_Week'}, inplace = True)
data.head(50)

,State,Atmospheric_Condition,Crash_Date,Crash_Fatalities,Roadway,Age,Alcohol_Results,Person_Type,Drug_Involvement,Race,Gender,Injury_Severity,Dayo_Week
0,Alaska,Clear,1/5/11,1,Rural-Principal Arterial-Interstate,27.0,0.00,Driver of a Motor Vehicle In-Transport,Yes,NaN,Male,Non-incapacitating Evident Injury (B),3
1,Alaska,Clear,1/5/11,1,Rural-Principal Arterial-Interstate,60.0,0.00,Driver of a Motor Vehicle In-Transport,No,White,Female,Fatal Injury (K),3
2,Arizona,Clear,1/1/11,1,Urban-Other Principal Arterial,24.0,NaN,Passenger of a Motor Vehicle In-Transport,Not Reported,NaN,Female,No Injury (O),6
3,Arizona,Clear,1/1/11,1,Urban-Other Principal Arterial,27.0,0.00,Driver of a Motor Vehicle In-Transport,No,NaN,Male,No Injury (O),6
4,Arizona,Clear,1/1/11,1,Urban-Other Principal Arterial,82.0,0.00,Pedestrian,No,Unknown,Female,Fatal Injury (K),6
5,Arkansas,Clear,1/2/11,1,Rural-Minor Arterial,40.0,0.00,Driver of a Motor Vehicle In-Transport,Not Reported,White,Male,Fatal Injury (K),7
6,Colorado,Clear,1/2/11,1,Rural-Local Road or Street,17.0,0.00,Driver of a Motor Vehicle In-Transport,No,White,Female,Fatal Injury (K),7
7,Connecticut,Clear,1/1/11,1,Urban-Local Road or Street,22.0,0.21,Driver of a Motor Vehicle In-Transport,Not Reported,White,Male,Fatal Injury (K),6
8,Delaware,Cloudy,1/4/11,1,Urban-Other Principal Arterial,4.0,NaN,Passenger of a Motor Vehicle In-Transport,Not Reported,NaN,Female,Non-incapacitating Evident Injury (B),2
9,Delaware,Cloudy,1/4/11,1,Urban-Other Principal Arterial,20.0,0.21,Driver of a Motor Vehicle In-Transport,Yes,White,Male,Fatal Injury (K),2


In [4]:
#study the data, starting with unique values for each column  
print(data.State.unique())
print(data.Atmospheric_Condition.unique())
print(data.Crash_Date.unique())
print(data.Crash_Fatalities.unique())
print(data.Roadway.unique())
print(data.Age.unique())
print(data.Alcohol_Results.unique())
print(data.Person_Type.unique())
print(data.Drug_Involvement.unique())
print(data.Race.unique())
print(data.Gender.unique())
print(data.Injury_Severity.unique())
print(data.Dayo_Week.unique())

['Alaska' 'Arizona' 'Arkansas' 'Colorado' 'Connecticut' 'Delaware'
 'District of Columbia' 'Florida' 'Georgia' 'Hawaii' 'Idaho' 'Illinois'
 'Indiana' 'Kansas' 'Kentucky' 'Louisiana' 'Maine' 'Massachusetts'
 'Michigan' 'Minnesota' 'Mississippi' 'Missouri' 'Montana' 'Nevada'
 'New Hampshire' 'New Mexico' 'New York' 'North Carolina' 'North Dakota'
 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania' 'South Carolina' 'Tennessee'
 'Texas' 'Utah' 'Vermont' 'Virginia' 'Washington' 'Wyoming' 'California'
 'Maryland' 'Nebraska' 'Alabama' 'Iowa' 'Rhode Island' 'South Dakota'
 'West Virginia' 'Wisconsin' 'New Jersey']
['Clear' 'Cloudy' 'Rain' 'Fog, Smog, Smoke' 'Snow' 'Blowing Snow'
 'Sleet, Hail (Freezing Rain or Drizzle)' 'Not Reported'
 'Blowing Sand, Soil, Dirt' 'Unknown' 'Severe Crosswinds' 'Other']
['1/5/11' '1/1/11' '1/2/11' '1/4/11' '1/14/11' '1/3/11' '1/25/11'
 '1/20/11' '2/12/11' '1/10/11' '1/7/11' '4/7/11' '2/1/11' '1/15/11'
 '1/16/11' '1/11/11' '1/22/11' '1/23/11' '1/30/11' '1/8/11' '1/18/11'
 

In [5]:
#Changing to na 
data.Atmospheric_Condition.replace('Not Reported', None, inplace = True)
data.Atmospheric_Condition.replace('Unknown', None, inplace = True)
data.Roadway.replace('\\N', None, inplace = True) 
data.Roadway.replace('Unknown', None, inplace = True) 
data.Drug_Involvement.replace('Not Reported', None, inplace = True)
data.Drug_Involvement.replace('\\N', None, inplace = True) 
data.Drug_Involvement.replace('Unknown', None, inplace = True) 
data.Race.replace('\\N', None, inplace = True)
data.Race.replace('Unknown', None, inplace = True)
data.Gender.replace('Not Reported', None, inplace = True)
data.Gender.replace('Unknown', None, inplace = True)
data.Injury_Severity.replace('Unknown', None, inplace = True)

In [6]:
print(data.State.unique())
print(data.Atmospheric_Condition.unique())
print(data.Crash_Date.unique())
print(data.Crash_Fatalities.unique())
print(data.Roadway.unique())
print(data.Age.unique())
print(data.Alcohol_Results.unique())
print(data.Person_Type.unique())
print(data.Drug_Involvement.unique())
print(data.Race.unique())
print(data.Gender.unique())
print(data.Injury_Severity.unique())
print(data.Dayo_Week.unique())

['Alaska' 'Arizona' 'Arkansas' 'Colorado' 'Connecticut' 'Delaware'
 'District of Columbia' 'Florida' 'Georgia' 'Hawaii' 'Idaho' 'Illinois'
 'Indiana' 'Kansas' 'Kentucky' 'Louisiana' 'Maine' 'Massachusetts'
 'Michigan' 'Minnesota' 'Mississippi' 'Missouri' 'Montana' 'Nevada'
 'New Hampshire' 'New Mexico' 'New York' 'North Carolina' 'North Dakota'
 'Ohio' 'Oklahoma' 'Oregon' 'Pennsylvania' 'South Carolina' 'Tennessee'
 'Texas' 'Utah' 'Vermont' 'Virginia' 'Washington' 'Wyoming' 'California'
 'Maryland' 'Nebraska' 'Alabama' 'Iowa' 'Rhode Island' 'South Dakota'
 'West Virginia' 'Wisconsin' 'New Jersey']
['Clear' 'Cloudy' 'Rain' 'Fog, Smog, Smoke' 'Snow' 'Blowing Snow'
 'Sleet, Hail (Freezing Rain or Drizzle)' 'Blowing Sand, Soil, Dirt'
 'Severe Crosswinds' 'Other']
['1/5/11' '1/1/11' '1/2/11' '1/4/11' '1/14/11' '1/3/11' '1/25/11'
 '1/20/11' '2/12/11' '1/10/11' '1/7/11' '4/7/11' '2/1/11' '1/15/11'
 '1/16/11' '1/11/11' '1/22/11' '1/23/11' '1/30/11' '1/8/11' '1/18/11'
 '1/6/11' '1/29/11' '1/12/

In [7]:
#Rename some values 

data.Atmospheric_Condition.replace('Hail (Freezing Rain or Drizzle)', 'Hail')

0       Clear
1       Clear
2       Clear
3       Clear
4       Clear
        ...  
4994    Clear
4995     Snow
4996    Clear
4997    Clear
4998     Rain
Name: Atmospheric_Condition, Length: 4999, dtype: object

In [8]:
data.loc[data['State'].isin(['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 
                         'Rhode Island', 'Vermont', 'New Jersey', 'New York', 'Pennsylvania']), 'Region'] = 'Northeast'
data.loc[data['State'].isin(['Illinois', 'Indiana', 'Michigan', 'Ohio', 'Wisconsin', 'Iowa', 
                        'Kansas', 'Minnesota', 'Missouri', 'Nebraska', 'North Dakota', 
                        'South Dakota']), 'Region'] = 'Midwest'
data.loc[data['State'].isin(['Delaware', 'Florida', 'Georgia', 'Maryland', 'North Carolina', 
                        'South Carolina', 'Virginia', 'District of Columbia', 'West Virginia', 'Alabama'
                        'Kentucky', 'Mississippi', 'Tennessee', 'Arkansas', 'Louisiana', 'Oklahoma', 'Texas']), 'Region'] = 'South'
data.loc[data['State'].isin(['Arizona', 'Colorado', 'Idaho', 'Montana','Nevada','New Mexico','Utah', 'Wyoming',
                        'Alaska','California','Hawaii','Oregon','Washington']), 'Region'] = 'West'

In [9]:
data = data[data.Region != 'Northeast']

data.head()

,State,Atmospheric_Condition,Crash_Date,Crash_Fatalities,Roadway,Age,Alcohol_Results,Person_Type,Drug_Involvement,Race,Gender,Injury_Severity,Dayo_Week,Region
0,Alaska,Clear,1/5/11,1,Rural-Principal Arterial-Interstate,27.0,0.0,Driver of a Motor Vehicle In-Transport,Yes,NaN,Male,Non-incapacitating Evident Injury (B),3,West
1,Alaska,Clear,1/5/11,1,Rural-Principal Arterial-Interstate,60.0,0.0,Driver of a Motor Vehicle In-Transport,No,White,Female,Fatal Injury (K),3,West
2,Arizona,Clear,1/1/11,1,Urban-Other Principal Arterial,24.0,NaN,Passenger of a Motor Vehicle In-Transport,No,NaN,Female,No Injury (O),6,West
3,Arizona,Clear,1/1/11,1,Urban-Other Principal Arterial,27.0,0.0,Driver of a Motor Vehicle In-Transport,No,NaN,Male,No Injury (O),6,West
4,Arizona,Clear,1/1/11,1,Urban-Other Principal Arterial,82.0,0.0,Pedestrian,No,NaN,Female,Fatal Injury (K),6,West


In [10]:
checkpoints = [0, 10, 20, 30, 40, 50, 60, 70, 80, 100]
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+']

data['age_range'] = pd.cut(data.Age, bins = checkpoints, labels = labels, include_lowest = True)

checkpoints2 = [0, 0.01, 0.5]
labels2 = ['No', 'Yes']

data['alcohol_range'] = pd.cut(data.Alcohol_Results, bins = checkpoints2, labels = labels2, include_lowest = True)

data.head()

,State,Atmospheric_Condition,Crash_Date,Crash_Fatalities,Roadway,Age,Alcohol_Results,Person_Type,Drug_Involvement,Race,Gender,Injury_Severity,Dayo_Week,Region,age_range,alcohol_range
0,Alaska,Clear,1/5/11,1,Rural-Principal Arterial-Interstate,27.0,0.0,Driver of a Motor Vehicle In-Transport,Yes,NaN,Male,Non-incapacitating Evident Injury (B),3,West,20-29,No
1,Alaska,Clear,1/5/11,1,Rural-Principal Arterial-Interstate,60.0,0.0,Driver of a Motor Vehicle In-Transport,No,White,Female,Fatal Injury (K),3,West,50-59,No
2,Arizona,Clear,1/1/11,1,Urban-Other Principal Arterial,24.0,NaN,Passenger of a Motor Vehicle In-Transport,No,NaN,Female,No Injury (O),6,West,20-29,NaN
3,Arizona,Clear,1/1/11,1,Urban-Other Principal Arterial,27.0,0.0,Driver of a Motor Vehicle In-Transport,No,NaN,Male,No Injury (O),6,West,20-29,No
4,Arizona,Clear,1/1/11,1,Urban-Other Principal Arterial,82.0,0.0,Pedestrian,No,NaN,Female,Fatal Injury (K),6,West,80+,No


In [11]:
set00 = data[['alcohol_range', 'Injury_Severity']]
set00_dummy = pd.get_dummies(set00)


DT = DecisionTreeClassifier(criterion = "entropy", random_state = 12345)

set00X = set00_dummy.iloc[:, :-6]
set00Y = set00_dummy.iloc[:, [-6]]

In [12]:
set00X_train, set00X_test, set00Y_train, set00Y_test = train_test_split(set00X, set00Y, test_size = 0.2, random_state = 12345) 

DT.fit(set00X_train, set00Y_train) 

DT_pred00 = DT.predict(set00X_test)

confusion_matrix(DT_pred00, set00Y_test)

DT_AS00 = accuracy_score(set00Y_test, DT_pred00) 

print(DT_AS00)

0.7417061611374408


In [13]:
set0 = data[['age_range', 'Injury_Severity']]
set0_dummy = pd.get_dummies(set0)


DT = DecisionTreeClassifier(criterion = "entropy", random_state = 12345)

set0X = set0_dummy.iloc[:, :-6]
set0Y = set0_dummy.iloc[:, [-6]]

In [14]:
set0X_train, set0X_test, set0Y_train, set0Y_test = train_test_split(set0X, set0Y, test_size = 0.2, random_state = 12345) 

DT.fit(set0X_train, set0Y_train) 

DT_pred0 = DT.predict(set0X_test)

confusion_matrix(DT_pred0, set0Y_test)

DT_AS0 = accuracy_score(set0Y_test, DT_pred0) 

print(DT_AS0)

0.5722748815165877


In [15]:
set1 = data[['Person_Type', 'Injury_Severity']]
set1_dummy = pd.get_dummies(set1)


DT = DecisionTreeClassifier(criterion = "entropy", random_state = 12345)

set1X = set1_dummy.iloc[:, :-6]
set1Y = set1_dummy.iloc[:, [-6]]

In [16]:
DT = DecisionTreeClassifier(criterion = "entropy", random_state = 12345)

set1X_train, set1X_test, set1Y_train, set1Y_test = train_test_split(set1X, set1Y, test_size = 0.2, random_state = 12345) 

DT.fit(set1X_train, set1Y_train) 

DT_pred = DT.predict(set1X_test)

confusion_matrix(DT_pred, set1Y_test)

DT_AS = accuracy_score(set1Y_test, DT_pred) 

print(DT_AS)

0.6054502369668247


In [17]:
Midwest = data.loc[data['Region'] == 'Midwest']
West = data.loc[data['Region'] == 'West']
South = data.loc[data['Region'] == 'South']

Overall_Set = data[['Person_Type', 'Region','alcohol_range', 'Injury_Severity']]
Midwest_Set = Midwest[['Person_Type', 'State', 'Injury_Severity']]
West_Set = West[['Person_Type', 'State','Injury_Severity']]
South_Set = South[['Person_Type', 'State','Injury_Severity']]

Overall_Set.to_csv('Overall.csv', index = False)
Midwest_Set.to_csv('Midwest.csv', index = False)
West_Set.to_csv('West.csv', index = False)
South_Set.to_csv('South.csv', index = False)

In [18]:
import arules
from arules.utils import five_quantile_based_bins, top_bottom_10, top_5_variant_variables
import csv

Overall_list=[]
MW_list=[]
S_list=[]
W_list = []
    
with open('Overall.csv') as x:
    reader=csv.reader(x, delimiter=',')
    for y in reader:
        print(y)
        Overall_list.append(y)    
    
with open('Midwest.csv') as x:
    reader=csv.reader(x, delimiter=',')
    for y in reader:
        print(y)
        MW_list.append(y)
        
with open('South.csv') as k:
    reader=csv.reader(k, delimiter=',')
    for t in reader:
        print(t)
        S_list.append(t)
        
with open('West.csv') as c:
    reader=csv.reader(c, delimiter=',')
    for d in reader:
        print(d)
        W_list.append(d)

['Person_Type', 'Region', 'alcohol_range', 'Injury_Severity']
['Driver of a Motor Vehicle In-Transport', 'West', 'No', 'Non-incapacitating Evident Injury (B)']
['Driver of a Motor Vehicle In-Transport', 'West', 'No', 'Fatal Injury (K)']
['Passenger of a Motor Vehicle In-Transport', 'West', '', 'No Injury (O)']
['Driver of a Motor Vehicle In-Transport', 'West', 'No', 'No Injury (O)']
['Pedestrian', 'West', 'No', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'South', 'No', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'West', 'No', 'Fatal Injury (K)']
['Passenger of a Motor Vehicle In-Transport', 'South', '', 'Non-incapacitating Evident Injury (B)']
['Driver of a Motor Vehicle In-Transport', 'South', 'Yes', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'South', '', 'Non-incapacitating Evident Injury (B)']
['Driver of a Motor Vehicle In-Transport', 'South', 'Yes', 'Fatal Injury (K)']
['Passenger of a Motor Vehicle In-Transport', 'South', 

['Passenger of a Motor Vehicle In-Transport', 'South', '', 'Non-incapacitating Evident Injury (B)']
['Passenger of a Motor Vehicle In-Transport', 'South', '', 'Non-incapacitating Evident Injury (B)']
['Passenger of a Motor Vehicle In-Transport', 'South', '', 'Incapacitating Injury (A)']
['Passenger of a Motor Vehicle In-Transport', 'West', '', 'No Injury (O)']
['Passenger of a Motor Vehicle In-Transport', 'West', 'No', 'Fatal Injury (K)']
['Passenger of a Motor Vehicle In-Transport', 'West', '', 'Non-incapacitating Evident Injury (B)']
['Passenger of a Motor Vehicle In-Transport', 'West', '', 'Non-incapacitating Evident Injury (B)']
['Driver of a Motor Vehicle In-Transport', 'West', 'Yes', 'Incapacitating Injury (A)']
['Passenger of a Motor Vehicle In-Transport', 'West', '', 'No Injury (O)']
['Driver of a Motor Vehicle In-Transport', 'West', 'No', 'Incapacitating Injury (A)']
['Driver of a Motor Vehicle In-Transport', 'West', 'No', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Tra

['Driver of a Motor Vehicle In-Transport', 'South', 'No', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'Midwest', 'Yes', 'Fatal Injury (K)']
['Passenger of a Motor Vehicle In-Transport', 'West', '', 'Possible Injury (C)']
['Passenger of a Motor Vehicle In-Transport', 'West', '', 'Possible Injury (C)']
['Passenger of a Motor Vehicle In-Transport', 'West', '', 'Possible Injury (C)']
['Passenger of a Motor Vehicle In-Transport', 'West', '', 'Incapacitating Injury (A)']
['Driver of a Motor Vehicle In-Transport', 'West', '', 'Non-incapacitating Evident Injury (B)']
['Passenger of a Motor Vehicle In-Transport', '', '', 'Incapacitating Injury (A)']
['Passenger of a Motor Vehicle In-Transport', '', '', 'Incapacitating Injury (A)']
['Driver of a Motor Vehicle In-Transport', '', 'No', 'Incapacitating Injury (A)']
['Driver of a Motor Vehicle In-Transport', '', 'No', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'West', 'No', 'Fatal Injury (K)']
['Pedestrian', 'W

['Driver of a Motor Vehicle In-Transport', 'South', 'Yes', 'Incapacitating Injury (A)']
['Passenger of a Motor Vehicle In-Transport', 'South', '', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'Midwest', 'No', 'No Injury (O)']
['Driver of a Motor Vehicle In-Transport', 'Midwest', 'No', 'Fatal Injury (K)']
['Passenger of a Motor Vehicle In-Transport', 'South', '', 'Non-incapacitating Evident Injury (B)']
['Passenger of a Motor Vehicle In-Transport', 'South', '', 'Incapacitating Injury (A)']
['Driver of a Motor Vehicle In-Transport', 'Midwest', 'Yes', 'Incapacitating Injury (A)']
['Driver of a Motor Vehicle In-Transport', 'Midwest', '', 'No Injury (O)']
['Passenger of a Motor Vehicle In-Transport', 'West', '', 'Fatal Injury (K)']
['Passenger of a Motor Vehicle In-Transport', 'West', '', 'Possible Injury (C)']
['Passenger of a Motor Vehicle In-Transport', '', '', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'West', '', 'No Injury (O)']
['Driver of a Moto

['Driver of a Motor Vehicle In-Transport', 'West', 'No', 'Fatal Injury (K)']
['Passenger of a Motor Vehicle In-Transport', 'West', '', 'Possible Injury (C)']
['Passenger of a Motor Vehicle In-Transport', 'West', 'No', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'West', '', 'Non-incapacitating Evident Injury (B)']
['Driver of a Motor Vehicle In-Transport', 'South', '', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'Midwest', 'No', 'No Injury (O)']
['Pedestrian', 'Midwest', 'No', 'Fatal Injury (K)']
['Pedestrian', 'Midwest', 'No', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'West', 'Yes', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'West', '', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'West', '', 'No Injury (O)']
['Pedestrian', 'West', 'No', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'West', 'Yes', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'South', '', '

...

['Pedestrian', 'South Carolina', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'South Carolina', 'No Injury (O)']
['Driver of a Motor Vehicle In-Transport', 'Tennessee', 'Fatal Injury (K)']
['Passenger of a Motor Vehicle In-Transport', 'Texas', 'Possible Injury (C)']
['Driver of a Motor Vehicle In-Transport', 'Texas', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'Texas', 'Incapacitating Injury (A)']
['Driver of a Motor Vehicle In-Transport', 'Virginia', 'Incapacitating Injury (A)']
['Passenger of a Motor Vehicle In-Transport', 'Virginia', 'Fatal Injury (K)']
['Passenger of a Motor Vehicle In-Transport', 'Georgia', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'Georgia', 'Incapacitating Injury (A)']
['Driver of a Motor Vehicle In-Transport', 'Mississippi', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'North Carolina', 'No Injury (O)']
['Driver of a Motor Vehicle In-Transport', 'Oklahoma', 'Fatal Injury (K)']
['Occupan

['Passenger of a Motor Vehicle In-Transport', 'Tennessee', 'No Injury (O)']
['Driver of a Motor Vehicle In-Transport', 'Virginia', 'Incapacitating Injury (A)']
['Driver of a Motor Vehicle In-Transport', 'Florida', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'Georgia', 'Fatal Injury (K)']
['Passenger of a Motor Vehicle In-Transport', 'North Carolina', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'North Carolina', 'Possible Injury (C)']
['Passenger of a Motor Vehicle In-Transport', 'North Carolina', 'Possible Injury (C)']
['Driver of a Motor Vehicle In-Transport', 'North Carolina', 'Possible Injury (C)']
['Driver of a Motor Vehicle In-Transport', 'South Carolina', 'Fatal Injury (K)']
['Passenger of a Motor Vehicle In-Transport', 'Texas', 'No Injury (O)']
['Driver of a Motor Vehicle In-Transport', 'Texas', 'No Injury (O)']
['Driver of a Motor Vehicle In-Transport', 'Texas', 'No Injury (O)']
['Driver of a Motor Vehicle In-Transport', 'Virginia', 'Fatal 

['Driver of a Motor Vehicle In-Transport', 'Florida', 'Fatal Injury (K)']
['Passenger of a Motor Vehicle In-Transport', 'Louisiana', 'No Injury (O)']
['Pedestrian', 'Louisiana', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'Maryland', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'Mississippi', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'North Carolina', 'No Injury (O)']
['Pedestrian', 'North Carolina', 'Fatal Injury (K)']
['Pedestrian', 'Oklahoma', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'South Carolina', 'Fatal Injury (K)']
['Passenger of a Motor Vehicle In-Transport', 'Tennessee', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'Tennessee', 'Fatal Injury (K)']
['Passenger of a Motor Vehicle In-Transport', 'Tennessee', 'Incapacitating Injury (A)']
['Driver of a Motor Vehicle In-Transport', 'Texas', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'Virginia', 'Fatal Injury (K)']


['Driver of a Motor Vehicle In-Transport', 'West Virginia', 'Fatal Injury (K)']
['Passenger of a Motor Vehicle In-Transport', 'Arkansas', 'Fatal Injury (K)']
['Passenger of a Motor Vehicle In-Transport', 'Delaware', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'Delaware', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'Delaware', 'No Injury (O)']
['Bicyclist', 'District of Columbia', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'Georgia', 'Non-incapacitating Evident Injury (B)']
['Driver of a Motor Vehicle In-Transport', 'Louisiana', 'Fatal Injury (K)']
['Passenger of a Motor Vehicle In-Transport', 'Louisiana', 'Incapacitating Injury (A)']
['Passenger of a Motor Vehicle In-Transport', 'Maryland', 'Incapacitating Injury (A)']
['Passenger of a Motor Vehicle In-Transport', 'Maryland', 'Incapacitating Injury (A)']
['Passenger of a Motor Vehicle In-Transport', 'Mississippi', 'No Injury (O)']
['Driver of a Motor Vehicle In-Transport', 'Sout

['Driver of a Motor Vehicle In-Transport', 'North Carolina', 'Incapacitating Injury (A)']
['Driver of a Motor Vehicle In-Transport', 'South Carolina', 'No Injury (O)']
['Driver of a Motor Vehicle In-Transport', 'South Carolina', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'Tennessee', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'Texas', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'Virginia', 'Fatal Injury (K)']
['Passenger of a Motor Vehicle In-Transport', 'West Virginia', 'Possible Injury (C)']
['Driver of a Motor Vehicle In-Transport', 'West Virginia', 'Possible Injury (C)']
['Driver of a Motor Vehicle In-Transport', 'Arkansas', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'Arkansas', 'No Injury (O)']
['Driver of a Motor Vehicle In-Transport', 'Florida', 'Possible Injury (C)']
['Passenger of a Motor Vehicle In-Transport', 'Louisiana', 'Fatal Injury (K)']
['Driver of a Motor Vehicle In-Transport', 'Louisiana', 

In [33]:
Overall_list = Overall_list[1:]
MW_list = MW_list[1:]
S_list = S_list[1:]
W_list = W_list[1:]

In [34]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [35]:
te = TransactionEncoder()
Overall_array = te.fit_transform(Overall_list)
Overall_freq = pd.DataFrame(Overall_array.astype("int"), columns=te.columns_)

Overall_items=apriori(Overall_freq, min_support=0.01, use_colnames=True)
Overall_items['length'] = Overall_items['itemsets'].apply(lambda x: len (x)) 

In [36]:
Overall_rules=association_rules(Overall_items, metric="confidence", min_threshold=0.8)

Overall_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Non-incapacitating Evident Injury (B)),(),0.140180,0.565465,0.115038,0.820643,1.451271,0.035771,2.422738
1,(Passenger of a Motor Vehicle In-Transport),(),0.321395,0.565465,0.283918,0.883395,1.562245,0.102181,3.726551
2,(No),(Driver of a Motor Vehicle In-Transport),0.304791,0.610294,0.254981,0.836576,1.370775,0.068969,2.384629
3,(Pedestrian),(Fatal Injury (K)),0.056452,0.463472,0.053131,0.941176,2.030706,0.026967,9.120968
4,(Yes),(Fatal Injury (K)),0.150854,0.463472,0.126660,0.839623,1.811591,0.056744,3.345407


In [37]:
Overall_rules.sort_values(by=['support', 'lift'], ascending = False, inplace = True)
Overall_rules[Overall_rules[['consequents', 'antecedents']].apply(lambda x: 'Fatal Injury (K)' in x.consequents and len(x.antecedents) ==1, axis=1 )]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(Yes),(Fatal Injury (K)),0.150854,0.463472,0.126660,0.839623,1.811591,0.056744,3.345407
3,(Pedestrian),(Fatal Injury (K)),0.056452,0.463472,0.053131,0.941176,2.030706,0.026967,9.120968


In [38]:
MW_array = te.fit_transform(MW_list)
MW_freq = pd.DataFrame(MW_array.astype("int"), columns=te.columns_)

MW_items=apriori(MW_freq, min_support=0.01, use_colnames=True)
MW_items['length'] = MW_items['itemsets'].apply(lambda x: len (x)) 

In [39]:
MW_rules=association_rules(MW_items, metric="confidence", min_threshold=0.6)

MW_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Fatal Injury (K)),(Driver of a Motor Vehicle In-Transport),0.469983,0.64837,0.328473,0.698905,1.077941,0.023750,1.167836
1,(Illinois),(Driver of a Motor Vehicle In-Transport),0.086621,0.64837,0.052316,0.603960,0.931505,-0.003847,0.887864
2,(Indiana),(Driver of a Motor Vehicle In-Transport),0.082333,0.64837,0.061750,0.750000,1.156746,0.008367,1.406518
3,(Iowa),(Driver of a Motor Vehicle In-Transport),0.072041,0.64837,0.051458,0.714286,1.101663,0.004749,1.230703
4,(Kansas),(Driver of a Motor Vehicle In-Transport),0.090909,0.64837,0.056604,0.622642,0.960317,-0.002339,0.931818


In [40]:
MW_rules.sort_values(by=['support', 'lift'], ascending = False, inplace = True)
MW_rules[MW_rules['consequents'].apply(lambda x: 'Fatal Injury (K)' in x )]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
11,(Pedestrian),(Fatal Injury (K)),0.041166,0.469983,0.037736,0.916667,1.950426,0.018388,6.360206
14,"(Driver of a Motor Vehicle In-Transport, Iowa)",(Fatal Injury (K)),0.051458,0.469983,0.032590,0.633333,1.347567,0.008406,1.445501


In [41]:
te = TransactionEncoder()
S_array = te.fit_transform(S_list)
S_freq = pd.DataFrame(S_array.astype("int"), columns=te.columns_)

S_items=apriori(S_freq, min_support=0.01, use_colnames=True)
S_items.head()

,support,itemsets
0,0.077470,(Arkansas)
1,0.075338,(Delaware)
2,0.034115,(District of Columbia)
3,0.625444,(Driver of a Motor Vehicle In-Transport)
4,0.477612,(Fatal Injury (K))


In [42]:
S_items['length'] = S_items['itemsets'].apply(lambda x: len (x)) 
S_rules=association_rules(S_items, metric="confidence", min_threshold=0.6)

S_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Arkansas),(Driver of a Motor Vehicle In-Transport),0.077470,0.625444,0.053305,0.688073,1.100136,0.004852,1.200782
1,(Delaware),(Driver of a Motor Vehicle In-Transport),0.075338,0.625444,0.045487,0.603774,0.965352,-0.001633,0.945307
2,(Fatal Injury (K)),(Driver of a Motor Vehicle In-Transport),0.477612,0.625444,0.334755,0.700893,1.120632,0.036035,1.252246
3,(Florida),(Driver of a Motor Vehicle In-Transport),0.072495,0.625444,0.044776,0.617647,0.987533,-0.000565,0.979607
4,(Georgia),(Driver of a Motor Vehicle In-Transport),0.057569,0.625444,0.038380,0.666667,1.065909,0.002373,1.123667


In [43]:
S_rules.sort_values(by=['support', 'lift'], ascending = False, inplace = True)
S_rules[S_rules['consequents'].apply(lambda x: 'Fatal Injury (K)' in x )]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
12,(Pedestrian),(Fatal Injury (K)),0.061123,0.477612,0.058991,0.965116,2.020712,0.029798,14.975124
11,(Georgia),(Fatal Injury (K)),0.057569,0.477612,0.034826,0.604938,1.266590,0.007330,1.322295
29,"(Driver of a Motor Vehicle In-Transport, West ...",(Fatal Injury (K)),0.048330,0.477612,0.029851,0.617647,1.293199,0.006768,1.366246
18,"(Georgia, Driver of a Motor Vehicle In-Transport)",(Fatal Injury (K)),0.038380,0.477612,0.027008,0.703704,1.473380,0.008677,1.763060
23,"(Driver of a Motor Vehicle In-Transport, Oklah...",(Fatal Injury (K)),0.034826,0.477612,0.022033,0.632653,1.324617,0.005399,1.422056


In [44]:
W_array = te.fit_transform(MW_list)
W_freq = pd.DataFrame(W_array.astype("int"), columns=te.columns_)

W_items=apriori(W_freq, min_support=0.01, use_colnames=True)
W_items.head()

,support,itemsets
0,0.648370,(Driver of a Motor Vehicle In-Transport)
1,0.469983,(Fatal Injury (K))
2,0.086621,(Illinois)
3,0.131218,(Incapacitating Injury (A))
4,0.082333,(Indiana)


In [45]:
W_items['length'] = W_items['itemsets'].apply(lambda x: len (x)) 

W_rules=association_rules(W_items, metric="confidence", min_threshold=0.6)
W_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Fatal Injury (K)),(Driver of a Motor Vehicle In-Transport),0.469983,0.64837,0.328473,0.698905,1.077941,0.023750,1.167836
1,(Illinois),(Driver of a Motor Vehicle In-Transport),0.086621,0.64837,0.052316,0.603960,0.931505,-0.003847,0.887864
2,(Indiana),(Driver of a Motor Vehicle In-Transport),0.082333,0.64837,0.061750,0.750000,1.156746,0.008367,1.406518
3,(Iowa),(Driver of a Motor Vehicle In-Transport),0.072041,0.64837,0.051458,0.714286,1.101663,0.004749,1.230703
4,(Kansas),(Driver of a Motor Vehicle In-Transport),0.090909,0.64837,0.056604,0.622642,0.960317,-0.002339,0.931818


In [46]:
W_rules.sort_values(by=['support', 'lift'], ascending = False, inplace = True)
W_rules[W_rules['consequents'].apply(lambda x: 'Fatal Injury (K)' in x )]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
11,(Pedestrian),(Fatal Injury (K)),0.041166,0.469983,0.037736,0.916667,1.950426,0.018388,6.360206
14,"(Driver of a Motor Vehicle In-Transport, Iowa)",(Fatal Injury (K)),0.051458,0.469983,0.032590,0.633333,1.347567,0.008406,1.445501
